In [1]:
import numpy as np
import pandas as pd

In [2]:
npz = np.load("data_mynp.npz", allow_pickle=True)

ranks = npz['ranks']
characteristics = npz['characteristics']
dates = npz['dates']
permnos = npz['permnos']
columns = npz['columns']
seq_valids = npz['seq_valids']
valids = npz['valids']
returns = npz['returns']

In [4]:
ranks.shape, permnos.shape, dates.shape

((653, 22873, 94), (22873,), (653,))

In [5]:
d = np.concatenate([ranks, permnos[None, :, None].repeat(ranks.shape[0], axis=0)], axis=2)

In [6]:
d = np.concatenate([d, dates[:, None, None].repeat(ranks.shape[1], axis=1)], axis=2)

In [7]:
d = d.reshape(-1, d.shape[2])

In [8]:
mask = (d[:, :94] == 0.0).all(axis=1)## < 10

In [9]:
mask.reshape(653, 22873).mean(axis=1)

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       0.99969396, 0.9995628 , 0.99877585, 0.99842609, 0.99790146,
       0.99750798, 0.99632755, 0.99252394, 0.98286189, 0.98238097,
       0.9819875 , 0.98181262, 0.98203122, 0.98220609, 0.98233725,
       0.98273073, 0.98273073, 0.98268701, 0.98531019, 0.98539763,
       0.98526647, 0.98557251, 0.98561623, 0.98561623, 0.98552879,
       0.98544135, 0.98513531, 0.98517903, 0.98517903, 0.98504787,
       0.98469812, 0.98395488, 0.98268701, 0.98264329, 0.98268701,
       0.98264329, 0.98246841, 0.98216237, 0.98163774, 0.98124426,
       0.98102566, 0.98085078, 0.97953919, 0.979102  , 0.97757181,
       0.97709089, 0.97669742, 0.97612906, 0.97551699, 0.97490491,
       0.97403052, 0.97319984, 0.97245661, 0.97070782, 0.96406243,
       0.95899095, 0.94775499, 0.94696804, 0.946662  , 0.94618109,
       0.94447602, 0.94408254, 0.94333931, 0.94259607, 0.94246491,
       0.9424212 , 0.94172168, 0.94115332, 0.9393171 , 0.93905

In [10]:
d = d[~mask]

In [11]:
d = np.concatenate([d, returns.reshape(-1, 1)[~mask]], axis=1)

In [12]:
df = pd.DataFrame(d, columns=columns.tolist() + ['permno', 'DATE', 'RET'])

In [13]:
for col in df.columns:
    if col == 'DATE':
        continue
    elif col == "permno":
        df[col] = df[col].astype(int)
    else:
        df[col] = df[col].astype(float)

In [14]:
df = df.sort_values(['permno', 'DATE'])

In [15]:
df.to_parquet("data_mynew.parquet")